
# Intro to LangChain

LangChain is an open-source framework that gives developers the tools they need to create applications using large language models (LLMs). In its essence, LangChain is a prompt orchestration tool that makes it easier for teams to connect various prompts interactively.

In [86]:
%pip install -qU openai
%pip install -qU langchain
%pip install -qU langchain-openai
%pip install -qU langchain-ollama
%pip install -qU tiktoken
%pip install -qU pydantic[email]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 313.6/313.6 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


# Accessing OpenAI Directly

In [15]:
import os
import openai

#os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ["OPENAI_API_KEY"]

client = openai.OpenAI()

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        ]
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              temperature=0.2
                                              )
    return response.choices[0].message.content

## Simple Query

In [16]:
response = get_completion("Can you tell me how much is 1+1?")
print(response)

1 + 1 equals 2.


## Queries with custom prompts using formatted strings

In [17]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [18]:
response = get_completion(prompt)
print(response)

I am quite frustrated that the lid of my blender came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I would appreciate your assistance with this issue. Thank you.


## Why should we use a framework to wrap the direct calls to OpenAI?    

The LangChain pipelines consist of the following modules:

+ **Models**: Models mostly cover Large Language models. A large language model of considerable size is a model that comprises a neural network with numerous parameters and is trained on vast quantities of unlabeled text  
+ **Prompts**: prompt is the input that we give to any system to refine our answers to make them more accurate or more specific according to our use case. Many times you may want to get more structured information than just text back. We can use Prompts in conjunction with Parsers
+ **Parsers**: Output parsers are responsible for taking the output of an LLM and transforming it to a more suitable format. This is very useful when you are using LLMs to generate any form of structured data.  
+ **Memory**: Chains and Agents in LangChain operate in a stateless mode by default, meaning that they handle each incoming query independently. However, there are certain applications, like chatbots, where it is of great importance to retain previous interactions, both over the short and long term. This is where the concept of “Memory” comes into play.  
+ **Chains**: Chains provide a means to merge various components into a unified application. A chain can be created, for instance, that receives input from a user, formats it using a PromptTemplate, and subsequently transmits the formatted reply to an LLM. More intricate chains can be generated by integrating multiple chains with other components.  
+ **Agents**: Certain applications may necessitate not only a pre-determined sequence of LLM/other tool calls but also an uncertain sequence that is dependent on the user’s input. These kinds of sequences include an “agent” that has access to a range of tools. Based on the user input, the agent may determine which of these tools, if any, should be called.  
+ **Callbacks**: Allow you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks.  
+ **Indexes**: Indexes are information stored in local databases that can be used for Augment the capabilities of the models being used. We will see them in action with RAG pipelines  

Let's wrap the pipeline:

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*05zEoeNU7DVYOFzjugiF_w.jpeg)


# I - Simple Pipelines with Langchain  

In [112]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import SystemMessage
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

import json
from pydantic import BaseModel, Field, model_validator, EmailStr, ValidationError
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.output_parsers import PydanticOutputParser

from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory

from langchain.chains import ConversationChain

In [63]:
#os.environ["OPENAI_API_KEY"] = "<the key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_ollama.llms import OllamaLLM

model1 = ChatOpenAI(model="gpt-3.5-turbo")
model2 = ChatOpenAI(model="gpt-4o")
model3 = ChatOpenAI(model="gpt-4o-mini", 
                   temperature=0)
model4 = OllamaLLM(model="gemma2:2b")

### Once you've installed and initialized the LLM of your choice, we could use it!   
### Let's ask it some question:

In [25]:
model3.invoke("Why is the sky blue?")

AIMessage(content="The sky appears blue primarily due to a phenomenon known as Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, which correspond to different wavelengths of light. Blue light has a shorter wavelength compared to other colors, such as red, which has a longer wavelength.\n\nAs sunlight passes through the atmosphere, it interacts with air molecules and small particles. The shorter wavelengths of light (blue and violet) are scattered in all directions more than the longer wavelengths (red, orange, yellow). Although violet light is scattered even more than blue light, our eyes are more sensitive to blue light, and some of the violet light is absorbed by the ozone layer, making the sky predominantly appear blue to us.\n\nDuring sunrise and sunset, the sky can appear red or orange because the sunlight has to pass through a thicker layer of the atmosphere. This means that the shorter blue wavelengths are scattered out of our l

In [26]:
model4.invoke("Why is the sky blue?")

"The blue color of the sky is a result of a fascinating phenomenon called **Rayleigh scattering**. Here's how it works:\n\n**1. Sunlight and Molecules:** \nSunlight, which appears white to our eyes, is actually made up of all the colors of the rainbow.  When this sunlight enters Earth's atmosphere, it interacts with tiny gas molecules (mainly nitrogen and oxygen) present in the air.\n\n**2. Scattering:** \nThese molecules scatter light in all directions when it hits them. However, blue light scatters more effectively than other colours due to a specific property of its wavelength.  Blue light has shorter wavelengths than red or orange light.\n\n**3. Blue Sky:** \nThe scattered blue light reaches our eyes from all directions across the sky, making us see a predominantly blue color. This is why we often see a clear, bright blue sky during the day.\n\n **Additional Points:**\n\n* **Time of Day:** At sunrise and sunset, the sunlight travels through a thicker layer of atmosphere.  This caus

### Prompt templates convert raw user input to better input to the LLM.

In [27]:
prompt = ChatPromptTemplate.from_messages([("system", "You are a first grade teacher."),
                                           ("user", "{input}")
                                         ])

chain = prompt | model4
chain.invoke({"input": "Why is the sky blue?"})

"That's a great question!  Did you know that the sky looks blue because of something called **sunlight** and **tiny things in the air**? \n\nImagine sunlight as tiny, bouncy balls. They hit the air and scatter all over the place. The blue light from the sun is like those bouncy balls bouncing around more than other colors -  so it gets scattered all around!  That's why we see the sky as blue most of the time. 🌈 \n\nBut sometimes the sky can look different, right? 🤔  It might be red or orange when the sun is setting because the sunlight has to travel a longer distance through more air.  And if there are clouds in the sky, they can block out the sunlight and make the sky appear gray!\n\n\nDo you have any other questions about why the sky looks so amazing? 😊 \n"

### The output of a ChatModel (and therefore, of this chain) is a message object. However, it's often much more convenient to work with strings.  
### Let's add a simple output parser to convert the chat message to a string.

In [28]:
output_parser = StrOutputParser()

In [29]:
chain = prompt | model4 | output_parser
chain.invoke({"input": "Why is the sky blue?"})

"Okay, great question!  You know how sometimes you see rainbows in the rain and sunlight shines so brightly through the water? 🌈☀️ Well, that's what makes the sky look blue too!\n\nHere's the secret: The sun's light is made up of all different colors - just like a rainbow! 🌈  But the air we breathe has tiny bits of stuff in it called molecules.  They are so small they let some colors through better than others. Blue and violet colors bounce around more easily because they are smaller! \n\nSo, when the sunlight shines on Earth's air, blue and violet light bounces around a lot more than the other colors, which makes us see the sky as blue! 💙  Pretty neat, huh? ✨\n\n\nDo you have any other questions about how things work in the world around us? 😊 \n"

# II - Exploring Chain Elements - Langchain Expression Language

Notice this line of the code, where we piece together these different components into a single chain using LCEL:  

**chain = prompt | model | output_parser**

## 1. [Prompts](https://python.langchain.com/docs/modules/model_io/prompts/)    
prompt is a BasePromptTemplate, which means it takes in a dictionary of template variables and produces a PromptValue.  
+ A PromptValue is a wrapper around a completed prompt that can be passed to either a LLM (which takes a string as input) or ChatModel (which takes a sequence of messages as input).  
+ It can work with either language model type because it defines logic both for producing BaseMessages and for producing a string.

In [30]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

chain = prompt | model4 | output_parser

chain.invoke({"topic": "chicken"})

'Why did the chicken cross the playground? \n\nTo get to the other slide! 😂 \n'

In [31]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream', additional_kwargs={}, response_metadata={})])

In [32]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream', additional_kwargs={}, response_metadata={})]

In [33]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [34]:
prompt = ChatPromptTemplate.from_messages([("system", "You are a first grade teacher."),
                                           ("user", "{input}")
                                         ])
chain = prompt | model3 | output_parser
chain.invoke({"input": "Why is the sky blue?"})

"The sky looks blue because of a process called Rayleigh scattering. When sunlight comes into the Earth's atmosphere, it is made up of many colors, like a rainbow. Blue light is scattered in all directions by the tiny particles in the air. Since blue light is scattered more than the other colors, we see a blue sky during the day! On sunrise and sunset, the light has to travel through more air, and the blue light scatters away, allowing us to see more reds and oranges. Isn't that cool?"

## 2. [Models](https://python.langchain.com/docs/modules/model_io/chat/quick_start/)    
#### The PromptValue is then passed to model.  
#### In this case our model is an OpenAI ChatModel, meaning it will output a BaseMessage.

In [37]:
message = model3.invoke(prompt_value)
print(message)
print(type(message))

content='What did the ice cream cone say to the scoop? \n\n"I’m sweet on you!" 🍦' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 15, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None} id='run-9a5e2128-09bb-448d-86ae-38736a840c41-0' usage_metadata={'input_tokens': 15, 'output_tokens': 20, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
<class 'langchain_core.messages.ai.AIMessage'>


#### If our model was an Ollama LLM, it would output a string.

In [38]:
message = model4.invoke(prompt_value)
print(message)
print(type(message))

Why did the ice cream cone go to the doctor? 

Because it was feeling crumby! 😂🍦 

<class 'str'>


#### That is why we should treat the model output with an output parser

## 3. [Output parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)    
And lastly we pass our model output to the output_parser, which is a BaseOutputParser meaning it takes either a string or a BaseMessage as input. The specific StrOutputParser simply converts any input into a string.

In [41]:
output_parser.invoke(message)

'Why did the ice cream cone go to the doctor? \n\nBecause it was feeling crumby! 😂🍦 \n'

In [42]:
print(type(message))

<class 'str'>


## 4. [Chains](https://python.langchain.com/v0.1/docs/modules/chains/)  

#### Chains are combination of steps. We have followed the steps along:

+ We passed the user input on the desired topic as {"topic": "ice cream"}
+ The prompt component takes the user input, which is then used to construct a PromptValue after using the topic to construct the prompt.
+ The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation.
+ The generated output from the model is a ChatMessage object.
+ Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.

#### Langchain has many types of chains using [LCEL](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel). We are just exploring some in this notebook. 

# 5. [Types of Prompts](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/)  
#### Let's explore some other types of Prompts  

## 5.1 PromptTemplate

Use PromptTemplate to create a template for a string prompt.  
By default, PromptTemplate uses Python’s str.format syntax for templating.

In [43]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [44]:
prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

## 5.2 ChatPromptTemplate

The prompt to chat models/ is a list of chat messages.  
Each chat message is associated with content, and an additional parameter called role. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [45]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

#### This is equivalent as doing the following, directly with OpenAI

In [46]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful AI bot. Your name is Bob."},
        {"role": "user", "content": "Hello, how are you doing?"},
        {"role": "assistant", "content": "I'm doing well, thanks!"},
        {"role": "user", "content": "What is your name?"},
    ],
)

The ChatPromptTemplate.from_messages static method accepts a variety of message representations and is a convenient way to format input to chat models with exactly the messages you want.  

For example, in addition to using the 2-tuple representation of (type, content) used above, you could pass in an instance of MessagePromptTemplate or BaseMessage.

In [47]:
chat_template = ChatPromptTemplate.from_messages(
    [SystemMessage(content=("You are a helpful assistant that re-writes the user's text to "
                            "sound more upbeat.")),
     HumanMessagePromptTemplate.from_template("{text}"),
    ])


messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I don't like eating tasty things", additional_kwargs={}, response_metadata={})]


## 5.3 Message Prompts  
LangChain provides different types of MessagePromptTemplate. The most commonly used are
+ AIMessagePromptTemplate  
+ SystemMessagePromptTemplate  
+ HumanMessagePromptTemplate  

Which create an AI message, system message and human message respectively.  
In cases where the chat model supports taking chat message with arbitrary role, you can use ChatMessagePromptTemplate, which allows user to specify the role name.  
https://python.langchain.com/docs/modules/model_io/chat/message_types/  

In [48]:
prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="Jedi", template=prompt)
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', additional_kwargs={}, response_metadata={}, role='Jedi')

# 5.4 MessagesPlaceholder  
LangChain also provides MessagesPlaceholder, which gives you full control of what messages to be rendered during formatting. This can be useful when you are uncertain of what role you should be using for your message prompt templates or when you wish to insert a list of messages during formatting.

In [49]:
human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)
chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"),
     human_message_template]
)

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.
2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, 
data types and control structures.
3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count="10").to_messages()

[HumanMessage(content='What is the best way to learn programming?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, \ndata types and control structures.\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Summarize our conversation so far in 10 words.', additional_kwargs={}, response_metadata={})]

# 6 - [Types of Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)  

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. Output parsers are classes that help structure language model responses.  
There are two main methods an output parser must implement:  
+ “Get format instructions”: A method which returns a string containing instructions for how the output of a language model should be formatted.
+ “Parse”: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

And then one optional one:

+ “Parse with prompt”: A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

#### Let's explore the **CSV Parser**

In [60]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(template="List five {subject}.\n{format_instructions}",
                        input_variables=["subject"],
                        partial_variables={"format_instructions": format_instructions},
)

chain = prompt | model4 | output_parser
a = chain.invoke({"subject": "ice cream flavors"})

In [61]:
print(a)
print(type(a))

['vanilla', 'chocolate', 'strawberry', 'mint chip', 'cookies and cream ']
<class 'list'>


In [62]:
for s in chain.stream({"subject": "ice cream flavors"}):
    print(s)

['Vanilla']
['chocolate']
['strawberry']
['mint chip']
['coffee ']


#### Let's take a look at the **Pandas Dataframe Parser**  

In [64]:
import pprint
from typing import Any, Dict
import pandas as pd

In [65]:
# Solely for documentation purposes.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

Define your desired Pandas DataFrame and set up a parser + inject instructions into the prompt template.

In [66]:

df = pd.DataFrame(
    {
        "num_legs": [2, 4, 8, 0],
        "num_wings": [2, 0, 0, 0],
        "num_specimen_seen": [10, 2, 1, 8],
    }
)

parser = PandasDataFrameOutputParser(dataframe=df)

Here's an example of a column operation being performed.

In [67]:
df_query = "Retrieve the num_wings column."

prompt = PromptTemplate(template="Answer the user query.\n{format_instructions}\n{query}\n",
                        input_variables=["query"],
                        partial_variables={"format_instructions": parser.get_format_instructions()},
                        )

chain = prompt | model4 | parser
parser_output = chain.invoke({"query": df_query})
format_parser_output(parser_output)

{'num_wings': {0: 2,
               1: 0,
               2: 0,
               3: 0}}


Here's an example of a row operation being performed.

In [68]:
df_query = "Retrieve the first row."

prompt = PromptTemplate(template="Answer the user query.\n{format_instructions}\n{query}\n",
                        input_variables=["query"],
                        partial_variables={"format_instructions": parser.get_format_instructions()},
                        )
chain = prompt | model4 | parser
parser_output = chain.invoke({"query": df_query})
format_parser_output(parser_output)

{'1': {'num_legs': 4,
       'num_specimen_seen': 2,
       'num_wings': 0}}


#### Below we go over a more powerful output parser, the **PydanticOutputParser**.

#### a) Define your desired data structure.

In [130]:
class UserInfo(BaseModel):
    name: str
    age: int
    email: EmailStr

#### b) Create a parser instance for the schema

In [143]:
parser = PydanticOutputParser(pydantic_object=UserInfo)

#### c) Create a prompt that formats the output in the required structure

In [ ]:
prompt_template = """
Extract the following information from the text:

name: The full name of the user.
age: The user's age (as an integer).
email: The user's email address.

Return the data in JSON format. 

Here's the text:
{text}
"""

#### d) Set up the Prompt Template

In [ ]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=prompt_template
)

# Example text input
text_input = "John Doe, a 30-year-old, can be reached at john.doe@example.com."

#### e) And a query intended to prompt a language model to populate the data structure.

In [139]:
prompt_and_model = prompt | model3
output = prompt_and_model.invoke({"text": text_input})
print(output.content)

```json
{
  "name": "John Doe",
  "age": 30,
  "email": "john.doe@example.com"
}
```


In [140]:
result = parser.invoke(output)
result

UserInfo(name='John Doe', age=30, email='john.doe@example.com')

In [141]:
result.json()

'{"name":"John Doe","age":30,"email":"john.doe@example.com"}'

### A more complex example

#### a) Define your desired data structure.

In [144]:
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values

#### b) Set up a parser and the prompt template.

In [145]:
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

#### c) And a query intended to prompt a language model to populate the data structure.

In [146]:
prompt_and_model = prompt | model3
output = prompt_and_model.invoke({"query": "Tell me a joke."})

result = parser.invoke(output)
result

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

In [147]:
result.json()

'{"setup":"Why don\'t scientists trust atoms?","punchline":"Because they make up everything!"}'

# 7 - [Memory](https://python.langchain.com/v0.1/docs/modules/memory/)  

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

### Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQcYZAK7bD8PPvjbAIe5tLk19SX9zKaSGHlVrE_vKrDk09tCgj7ujp_re7SpYHI8I7yUA&usqp=CAU) 

## 7.1 - ConversationBufferMemory  

We are going to use the simplest memory type.  
Let's check what is happening behind the scenes using the ```verbose``` mode of the LLM:  

In [149]:
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=model3, memory=memory, verbose=True)
conversation.predict(input="Hi, my name is Renato")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Renato
AI:

> Finished chain.


"Hello, Renato! It's great to meet you! How's your day going so far?"

In [150]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Renato
AI: Hello, Renato! It's great to meet you! How's your day going so far?
Human: What is 1+1?
AI:

> Finished chain.


"1 + 1 equals 2! It's a simple addition problem, but it's the foundation of so many mathematical concepts. Do you enjoy math, or is there another subject you prefer?"

In [151]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Renato
AI: Hello, Renato! It's great to meet you! How's your day going so far?
Human: What is 1+1?
AI: 1 + 1 equals 2! It's a simple addition problem, but it's the foundation of so many mathematical concepts. Do you enjoy math, or is there another subject you prefer?
Human: What is my name?
AI:

> Finished chain.


"Your name is Renato! It's a nice name. Do you have any favorite hobbies or interests you'd like to share?"

#### As we are using a memory module, we can check what is stored in there:  

In [152]:
print(memory.buffer)

Human: Hi, my name is Renato
AI: Hello, Renato! It's great to meet you! How's your day going so far?
Human: What is 1+1?
AI: 1 + 1 equals 2! It's a simple addition problem, but it's the foundation of so many mathematical concepts. Do you enjoy math, or is there another subject you prefer?
Human: What is my name?
AI: Your name is Renato! It's a nice name. Do you have any favorite hobbies or interests you'd like to share?


In [153]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Renato\nAI: Hello, Renato! It's great to meet you! How's your day going so far?\nHuman: What is 1+1?\nAI: 1 + 1 equals 2! It's a simple addition problem, but it's the foundation of so many mathematical concepts. Do you enjoy math, or is there another subject you prefer?\nHuman: What is my name?\nAI: Your name is Renato! It's a nice name. Do you have any favorite hobbies or interests you'd like to share?"}

#### Now let's erase the memory buffer and insert something else in it:  

In [154]:
memory = ConversationBufferMemory()

memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

print(memory.buffer)

Human: Hi
AI: What's up


In [155]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

#### Let's add even more context:  

In [156]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

##### Now, let's check if the model still know the information from a previous interaction:  

In [157]:
conversation = ConversationChain(llm=model3,
                                 memory=memory,
                                 verbose=True
                                 )
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What is my name?
AI:

> Finished chain.


"I don't know your name yet! But I'd love to learn it if you'd like to share. What should I call you?"

## ConversationBufferWindowMemory  
[See changes](https://python.langchain.com/docs/versions/migrating_memory/)

We can define a buffer size to determine how much context will be kept:  

In [158]:
memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.load_memory_variables({})

C:\Users\Renato Rocha Souza\AppData\Local\Temp\ipykernel_11776\1737933177.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


{'history': 'Human: Not much, just hanging\nAI: Cool'}

#### We see that only one interaction was saved. Now let's see how it would be with the LLM:  

In [159]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=model, memory=memory, verbose=False)
conversation.predict(input="Hi, my name is Renato")

"Hello Renato! It's great to meet you! I'm here to chat about anything you'd like. How's your day going so far?"

In [160]:
conversation.predict(input="What is 1+1?")

"1 + 1 equals 2! It's one of the simplest math problems out there. Do you enjoy math, or is there another subject that interests you more?"

In [161]:
conversation.predict(input="What is my name?")

"I don't have access to your name unless you tell me! But I’d love to know what it is. What's your name?"

## ConversationTokenBufferMemory

Similarly, we can define a buffer size in the quantity of tokens:  

In [162]:
memory = ConversationTokenBufferMemory(llm=model, max_token_limit=30)

memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})

memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Something Beautiful!"})

memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

memory.load_memory_variables({})

C:\Users\Renato Rocha Souza\AppData\Local\Temp\ipykernel_11776\3519355949.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(llm=model, max_token_limit=30)


{'history': 'AI: Something Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory  

This last type of memory creates an automatic summary of the previous interactions to store:  

In [163]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=model, max_token_limit=100)

memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

memory.load_memory_variables({})

C:\Users\Renato Rocha Souza\AppData\Local\Temp\ipykernel_11776\1480727767.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=model, max_token_limit=100)


{'history': "System: The human greets the AI, and they exchange casual pleasantries. The human then inquires about the day's schedule.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo."}

In [164]:
conversation = ConversationChain(llm=model, memory=memory, verbose=True)
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI, and they exchange casual pleasantries. The human then inquires about the day's schedule.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A great demo to showcase would be the capabilities of a conversational AI model, like a chatbot that can answer questions, assist with tasks, or engage in casual conversation. You could create a scenario where the chatbot helps a user book a restaurant reservation or provides recommendations based on their preferences. Another impressive option could be demonstrating how the model generates text based on a prompt, such as writing a short story or creating a marketing email. If you have access to a tool like LangChain, you could also demonstrate how it integrates different data sources or APIs to provide a more dynamic and interactive experience. This would really highlight the versatility and real-world applications of LLMs!'

In [165]:
memory.load_memory_variables({})

{'history': "System: The human greets the AI, and they exchange casual pleasantries. The human then inquires about the day's schedule, to which the AI responds with details about meetings and tasks, including a product team meeting at 8 am, time to work on a LangChain project, and a lunch appointment with a customer at noon. The human asks for a good demo to show, and the AI suggests showcasing a conversational AI model, like a chatbot that can assist with tasks or engage in conversations, as well as demonstrating text generation capabilities. The AI emphasizes using LangChain to integrate different data sources or APIs to highlight the versatility and real-world applications of LLMs."}